# 5.1.3 dlib Integration - 68點面部特徵檢測與人臉識別

本模組介紹如何使用dlib函式庫進行高精度的人臉檢測、68點面部特徵定位和人臉識別。

## 學習目標
- 掌握dlib人臉檢測器的使用
- 實現68點面部特徵點檢測
- 進行人臉對齊和標準化
- 實現基於深度學習的人臉識別
- 比較不同人臉檢測方法的效能

## 1. 環境設置與函式庫導入

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

# 添加utils路徑
sys.path.append('../utils')
from image_utils import load_image, resize_image
from visualization import display_image, display_multiple_images
from performance import time_function, benchmark_function

# 嘗試導入dlib
try:
    import dlib
    DLIB_AVAILABLE = True
    print("✅ dlib library loaded successfully")
    print(f"dlib version: {dlib.version}")
except ImportError:
    DLIB_AVAILABLE = False
    print("⚠️ dlib not available. Please install: pip install dlib")
    print("Some features will be disabled.")

# 設置matplotlib中文顯示
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. dlib人臉檢測器初始化

In [ ]:
def initialize_dlib_detectors():
    """
    初始化dlib檢測器和預測器
    
    Returns:
        tuple: (人臉檢測器, 68點預測器)
    """
    if not DLIB_AVAILABLE:
        return None, None
    
    # 初始化HOG人臉檢測器
    face_detector = dlib.get_frontal_face_detector()
    
    # 68點面部特徵預測器路徑
    predictor_path = "../assets/models/shape_predictor_68_face_landmarks.dat"
    
    if os.path.exists(predictor_path):
        landmark_predictor = dlib.shape_predictor(predictor_path)
        print(f"✅ 68-point landmark predictor loaded from {predictor_path}")
    else:
        print(f"⚠️ Landmark predictor not found at {predictor_path}")
        print("Please download from: http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2")
        landmark_predictor = None
    
    return face_detector, landmark_predictor

# 初始化檢測器
detector, predictor = initialize_dlib_detectors()

## 3. dlib人臉檢測功能

In [ ]:
def detect_faces_dlib(image, upsampling=1):
    """
    使用dlib檢測人臉
    
    Args:
        image: 輸入圖像
        upsampling: 上採樣次數，增加檢測精度但降低速度
    
    Returns:
        list: 檢測到的人臉矩形框列表
    """
    if not DLIB_AVAILABLE or detector is None:
        return []
    
    # 轉換為灰階
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    # 檢測人臉
    faces = detector(gray, upsampling)
    
    # 轉換為OpenCV格式 (x, y, w, h)
    face_rects = []
    for face in faces:
        x = face.left()
        y = face.top()
        w = face.width()
        h = face.height()
        face_rects.append((x, y, w, h))
    
    return face_rects

@time_function
def detect_faces_dlib_timed(image, upsampling=1):
    """帶計時的dlib人臉檢測"""
    return detect_faces_dlib(image, upsampling)

# 測試dlib人臉檢測
def test_dlib_face_detection():
    """測試dlib人臉檢測功能"""
    test_image_path = "../assets/images/basic/faces01.jpg"
    
    if not os.path.exists(test_image_path):
        print(f"測試圖片不存在: {test_image_path}")
        return
    
    image = load_image(test_image_path)
    if image is None:
        return
    
    # 調整圖片大小以提高檢測速度
    image_resized = resize_image(image, max_width=800)
    
    # dlib檢測
    faces_dlib = detect_faces_dlib_timed(image_resized)
    
    # 繪製檢測結果
    result_dlib = image_resized.copy()
    for (x, y, w, h) in faces_dlib:
        cv2.rectangle(result_dlib, (x, y), (x+w, y+h), (0, 255, 0), 3)
        cv2.putText(result_dlib, 'dlib HOG', (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    print(f"\n✅ dlib檢測結果: 發現 {len(faces_dlib)} 張人臉")
    
    # 顯示結果
    display_multiple_images(
        [image_resized, result_dlib],
        ["原始圖片", f"dlib檢測 ({len(faces_dlib)} faces)"],
        figsize=(12, 6)
    )

# 執行測試
test_dlib_face_detection()

## 4. 68點面部特徵檢測

In [ ]:
def detect_facial_landmarks(image, face_rect):
    """
    檢測68點面部特徵
    
    Args:
        image: 輸入圖像
        face_rect: 人臉矩形框 (x, y, w, h)
    
    Returns:
        np.array: 68個特徵點座標 (68, 2)
    """
    if not DLIB_AVAILABLE or predictor is None:
        return np.array([])
    
    # 轉換為灰階
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image
    
    # 轉換為dlib矩形格式
    x, y, w, h = face_rect
    dlib_rect = dlib.rectangle(x, y, x + w, y + h)
    
    # 檢測特徵點
    landmarks = predictor(gray, dlib_rect)
    
    # 轉換為NumPy陣列
    points = np.array([(landmarks.part(i).x, landmarks.part(i).y) 
                      for i in range(68)])
    
    return points

def draw_facial_landmarks(image, landmarks, show_numbers=False):
    """
    繪製68點面部特徵
    
    Args:
        image: 輸入圖像
        landmarks: 68個特徵點座標
        show_numbers: 是否顯示點的編號
    
    Returns:
        np.array: 繪製特徵點後的圖像
    """
    result = image.copy()
    
    if len(landmarks) == 0:
        return result
    
    # 定義不同區域的顏色
    colors = {
        'jaw': (255, 0, 0),      # 下巴輪廓 (0-16)
        'eyebrow_r': (0, 255, 0), # 右眉毛 (17-21)
        'eyebrow_l': (0, 255, 0), # 左眉毛 (22-26)
        'nose': (0, 0, 255),     # 鼻子 (27-35)
        'eye_r': (255, 255, 0),  # 右眼 (36-41)
        'eye_l': (255, 255, 0),  # 左眼 (42-47)
        'mouth': (255, 0, 255)   # 嘴巴 (48-67)
    }
    
    # 定義各部位的點範圍
    regions = {
        'jaw': range(0, 17),
        'eyebrow_r': range(17, 22),
        'eyebrow_l': range(22, 27),
        'nose': range(27, 36),
        'eye_r': range(36, 42),
        'eye_l': range(42, 48),
        'mouth': range(48, 68)
    }
    
    # 繪製特徵點
    for region, point_range in regions.items():
        color = colors[region]
        for i in point_range:
            if i < len(landmarks):
                x, y = landmarks[i]
                cv2.circle(result, (int(x), int(y)), 3, color, -1)
                
                if show_numbers:
                    cv2.putText(result, str(i), (int(x) + 3, int(y) - 3),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.3, color, 1)
    
    return result

# 測試68點特徵檢測
def test_facial_landmarks():
    """測試68點面部特徵檢測"""
    test_image_path = "../assets/images/basic/face03.jpg"
    
    if not os.path.exists(test_image_path):
        print(f"測試圖片不存在: {test_image_path}")
        return
    
    image = load_image(test_image_path)
    if image is None:
        return
    
    # 調整圖片大小
    image_resized = resize_image(image, max_width=600)
    
    # 檢測人臉
    faces = detect_faces_dlib(image_resized)
    
    if len(faces) == 0:
        print("未檢測到人臉")
        return
    
    # 對每個檢測到的人臉進行特徵點檢測
    results = []
    titles = []
    
    for i, face_rect in enumerate(faces):
        # 檢測68點特徵
        landmarks = detect_facial_landmarks(image_resized, face_rect)
        
        if len(landmarks) > 0:
            # 繪製特徵點（不顯示編號）
            result_landmarks = draw_facial_landmarks(image_resized, landmarks, False)
            
            # 繪製人臉框
            x, y, w, h = face_rect
            cv2.rectangle(result_landmarks, (x, y), (x+w, y+h), (0, 255, 0), 2)
            
            results.append(result_landmarks)
            titles.append(f"68點特徵檢測 - 人臉{i+1}")
            
            print(f"✅ 人臉{i+1}: 成功檢測68個特徵點")
    
    # 顯示結果
    if results:
        all_images = [image_resized] + results
        all_titles = ["原始圖片"] + titles
        
        display_multiple_images(all_images, all_titles, figsize=(15, 5))

# 執行測試
test_facial_landmarks()

## 5. 人臉對齊功能

In [ ]:
def align_face(image, landmarks, desired_size=(256, 256)):
    """
    基於眼部特徵點對齊人臉
    
    Args:
        image: 輸入圖像
        landmarks: 68個特徵點
        desired_size: 輸出圖像大小
    
    Returns:
        np.array: 對齊後的人臉圖像
    """
    if len(landmarks) == 0:
        return None
    
    # 取得左眼和右眼的中心點
    # 左眼: 點42-47, 右眼: 點36-41
    left_eye_pts = landmarks[42:48]
    right_eye_pts = landmarks[36:42]
    
    left_eye_center = np.mean(left_eye_pts, axis=0)
    right_eye_center = np.mean(right_eye_pts, axis=0)
    
    # 計算眼部中線的角度
    dy = right_eye_center[1] - left_eye_center[1]
    dx = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dy, dx))
    
    # 計算兩眼之間的距離
    eye_distance = np.sqrt(dx**2 + dy**2)
    
    # 計算縮放比例（假設理想的眼距為臉部寬度的40%）
    desired_eye_distance = desired_size[0] * 0.4
    scale = desired_eye_distance / eye_distance
    
    # 計算旋轉中心（兩眼中點）
    eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                   (left_eye_center[1] + right_eye_center[1]) // 2)
    
    # 獲得旋轉矩陣
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)
    
    # 調整平移以將臉部置中
    tx = desired_size[0] * 0.5
    ty = desired_size[1] * 0.4  # 眼部位置稍微偏上
    
    M[0, 2] += (tx - eyes_center[0])
    M[1, 2] += (ty - eyes_center[1])
    
    # 執行仿射變換
    aligned_face = cv2.warpAffine(image, M, desired_size)
    
    return aligned_face

# 測試人臉對齊功能
def test_face_alignment():
    """測試人臉對齊功能"""
    test_image_path = "../assets/images/basic/face03.jpg"
    
    if not os.path.exists(test_image_path):
        print(f"測試圖片不存在: {test_image_path}")
        return
    
    image = load_image(test_image_path)
    if image is None:
        return
    
    # 調整圖片大小
    image_resized = resize_image(image, max_width=600)
    
    # 檢測人臉
    faces = detect_faces_dlib(image_resized)
    
    if len(faces) == 0:
        print("未檢測到人臉")
        return
    
    results = [image_resized]
    titles = ["原始圖片"]
    
    for i, face_rect in enumerate(faces):
        # 檢測特徵點
        landmarks = detect_facial_landmarks(image_resized, face_rect)
        
        if len(landmarks) > 0:
            # 對齊人臉
            aligned_face = align_face(image_resized, landmarks, (200, 200))
            
            if aligned_face is not None:
                results.append(aligned_face)
                titles.append(f"對齊人臉{i+1}")
                print(f"✅ 人臉{i+1}: 對齊成功")
    
    # 顯示結果
    display_multiple_images(results, titles, figsize=(12, 4))

# 執行測試
test_face_alignment()

## 6. 人臉檢測方法比較

In [ ]:
def compare_face_detection_methods(image_path):
    """
    比較不同人臉檢測方法的效能
    
    Args:
        image_path: 測試圖像路徑
    """
    if not os.path.exists(image_path):
        print(f"圖片不存在: {image_path}")
        return
    
    image = load_image(image_path)
    if image is None:
        return
    
    # 調整圖片大小以確保公平比較
    image_resized = resize_image(image, max_width=800)
    gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    
    results = []
    titles = []
    detection_times = []
    face_counts = []
    
    # 1. Haar Cascade檢測
    try:
        haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 
                                            'haarcascade_frontalface_default.xml')
        
        @time_function
        def detect_haar():
            return haar_cascade.detectMultiScale(gray, 1.1, 5, minSize=(30, 30))
        
        faces_haar, time_haar = detect_haar()
        
        # 繪製結果
        result_haar = image_resized.copy()
        for (x, y, w, h) in faces_haar:
            cv2.rectangle(result_haar, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        results.append(result_haar)
        titles.append(f"Haar Cascade\n{len(faces_haar)} faces, {time_haar:.1f}ms")
        detection_times.append(time_haar)
        face_counts.append(len(faces_haar))
        
    except Exception as e:
        print(f"Haar Cascade檢測失敗: {e}")
    
    # 2. dlib HOG檢測
    if DLIB_AVAILABLE:
        faces_dlib, time_dlib = detect_faces_dlib_timed(image_resized)
        
        # 繪製結果
        result_dlib = image_resized.copy()
        for (x, y, w, h) in faces_dlib:
            cv2.rectangle(result_dlib, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        results.append(result_dlib)
        titles.append(f"dlib HOG\n{len(faces_dlib)} faces, {time_dlib:.1f}ms")
        detection_times.append(time_dlib)
        face_counts.append(len(faces_dlib))
    
    # 3. OpenCV DNN檢測 (如果模型存在)
    dnn_model_path = "../assets/models/opencv_face_detector_uint8.pb"
    dnn_config_path = "../assets/models/opencv_face_detector.pbtxt"
    
    if os.path.exists(dnn_model_path) and os.path.exists(dnn_config_path):
        try:
            @time_function
            def detect_dnn():
                net = cv2.dnn.readNetFromTensorflow(dnn_model_path, dnn_config_path)
                h, w = image_resized.shape[:2]
                blob = cv2.dnn.blobFromImage(image_resized, 1.0, (300, 300), [104, 117, 123])
                net.setInput(blob)
                detections = net.forward()
                
                faces = []
                for i in range(detections.shape[2]):
                    confidence = detections[0, 0, i, 2]
                    if confidence > 0.5:
                        x1 = int(detections[0, 0, i, 3] * w)
                        y1 = int(detections[0, 0, i, 4] * h)
                        x2 = int(detections[0, 0, i, 5] * w)
                        y2 = int(detections[0, 0, i, 6] * h)
                        faces.append((x1, y1, x2-x1, y2-y1))
                
                return faces
            
            faces_dnn, time_dnn = detect_dnn()
            
            # 繪製結果
            result_dnn = image_resized.copy()
            for (x, y, w, h) in faces_dnn:
                cv2.rectangle(result_dnn, (x, y), (x+w, y+h), (0, 0, 255), 2)
            
            results.append(result_dnn)
            titles.append(f"OpenCV DNN\n{len(faces_dnn)} faces, {time_dnn:.1f}ms")
            detection_times.append(time_dnn)
            face_counts.append(len(faces_dnn))
            
        except Exception as e:
            print(f"OpenCV DNN檢測失敗: {e}")
    
    # 顯示比較結果
    if results:
        all_images = [image_resized] + results
        all_titles = ["原始圖片"] + titles
        
        display_multiple_images(all_images, all_titles, figsize=(15, 5))
        
        # 輸出性能統計
        print("\n📊 人臉檢測方法比較:")
        print("-" * 50)
        methods = ["Haar Cascade", "dlib HOG", "OpenCV DNN"][:len(detection_times)]
        for i, method in enumerate(methods):
            print(f"{method:12}: {face_counts[i]:2d} faces, {detection_times[i]:6.1f}ms")

# 執行比較測試
test_image_path = "../assets/images/basic/faces01.jpg"
compare_face_detection_methods(test_image_path)

## 7. 實作練習

### 練習1: 批量人臉特徵提取

In [ ]:
def batch_face_feature_extraction(image_folder, output_folder):
    """
    批量提取資料夾中所有圖片的人臉特徵
    
    練習目標:
    1. 遍歷資料夾中的所有圖片
    2. 檢測每張圖片中的人臉
    3. 提取68點特徵並儲存結果
    4. 生成對齊後的人臉圖片
    
    請完成以下功能:
    - 檢查輸入/輸出資料夾是否存在
    - 處理不同格式的圖片檔案
    - 錯誤處理（沒有檢測到人臉的情況）
    - 進度顯示
    """
    # TODO: 實作批量人臉特徵提取
    pass

print("💡 練習1: 請實作批量人臉特徵提取功能")
print("提示: 使用os.listdir()遍歷檔案，用try-except處理錯誤")

### 練習2: 人臉相似度計算

In [ ]:
def calculate_face_similarity(landmarks1, landmarks2):
    """
    計算兩個人臉特徵點的相似度
    
    練習目標:
    1. 正規化特徵點座標
    2. 計算歐幾里得距離
    3. 實現相似度評分
    
    提示:
    - 使用特徵點之間的相對距離而非絕對座標
    - 考慮使用普氏分析 (Procrustes Analysis)
    - 回傳0-1之間的相似度分數
    """
    # TODO: 實作人臉相似度計算
    pass

print("💡 練習2: 請實作人臉相似度計算功能")
print("提示: 可以使用numpy.linalg.norm計算距離")

## 8. 總結與延伸應用

### 本模組重點回顧

1. **dlib人臉檢測**: HOG + SVM方法，高精度但速度較慢
2. **68點特徵檢測**: 精確的面部特徵定位，支援人臉分析
3. **人臉對齊**: 基於眼部特徵的幾何校正
4. **方法比較**: Haar vs dlib vs DNN的效能分析

### 實際應用場景
- 人臉識別系統
- 表情分析
- 美顏相機濾鏡
- 人臉動畫驅動
- 生物特徵認證

### 下一步學習
- 5.2.1 深度學習物體檢測 (YOLO, SSD)
- 5.2.2 實時檢測優化
- 人臉識別與驗證算法
- 實戰專案：智能監控系統

### 效能基準測試

In [ ]:
# 最終效能測試
def run_performance_benchmark():
    """執行完整的效能基準測試"""
    
    test_cases = [
        "../assets/images/basic/face03.jpg",
        "../assets/images/basic/faces01.jpg",
        "../assets/images/basic/faces.png"
    ]
    
    print("🔄 執行dlib效能基準測試...\n")
    
    for i, image_path in enumerate(test_cases, 1):
        if os.path.exists(image_path):
            print(f"測試{i}: {os.path.basename(image_path)}")
            image = load_image(image_path)
            if image is not None:
                image_resized = resize_image(image, max_width=600)
                
                # 檢測並計時
                faces, detection_time = detect_faces_dlib_timed(image_resized)
                print(f"  - 檢測時間: {detection_time:.1f}ms")
                print(f"  - 人臉數量: {len(faces)}")
                
                # 特徵點檢測
                if len(faces) > 0:
                    landmarks = detect_facial_landmarks(image_resized, faces[0])
                    if len(landmarks) > 0:
                        print(f"  - 特徵點: ✅ 68點檢測成功")
                    else:
                        print(f"  - 特徵點: ❌ 檢測失敗")
                print()
    
    print("✅ dlib整合模組測試完成")

# 執行基準測試
run_performance_benchmark()